In [2]:
import pandas as pd
import xmlrpc.client

In [3]:
import xmlrpc.client
import pandas as pd

# Conexión con Odoo
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")




fields_info = models.execute_kw(
    db, uid, password,
    'account.invoice.report',  # Nombre técnico del modelo
    'fields_get',
    []
)

# Convertir a DataFrame para visualizar mejor
df_campos = pd.DataFrame.from_dict(fields_info, orient='index')
df_campos

,sortable,string,searchable,required,manual,readonly,states,depends,groups,company_dependent,type,store,relation,digits,translate,size,selection,domain,help
date_due,True,Due Date,True,False,False,True,{},[],False,False,date,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
partner_bank_id,True,Bank Account,True,False,False,True,{},[],False,False,many2one,True,res.partner.bank,NaN,NaN,NaN,NaN,NaN,NaN
payment_term,True,Payment Term,True,False,False,True,{},[],False,False,many2one,True,account.payment.term,NaN,NaN,NaN,NaN,NaN,NaN
nbr,True,# of Invoices,True,False,False,True,{},[],False,False,integer,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
journal_id,True,Journal,True,False,False,True,{},[],False,False,many2one,True,account.journal,NaN,NaN,NaN,NaN,NaN,NaN
currency_id,True,Currency,True,False,False,True,{},[],False,False,many2one,True,res.currency,NaN,NaN,NaN,NaN,NaN,NaN
product_qty,True,Product Quantity,True,False,False,True,{},[],False,False,float,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_currency_price_average,False,Average Price,False,False,False,True,{},[],False,False,float,False,NaN,"[16, 2]",NaN,NaN,NaN,NaN,NaN
id,True,Id,True,False,False,True,{},[],False,False,integer,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
account_id,True,Account,True,False,False,True,{},[],False,False,many2one,True,account.account,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
import xmlrpc.client
import pandas as pd
from datetime import datetime

# ---------------- Conexión ----------------
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# ---------------- Campos a extraer ----------------
campos = [
    'product_marca',
    'product_cost',
    'price_total',
    'date'
]

# ---------------- Años a procesar ----------------
anios = range(2025, 2026)  # 2020–2025 inclusive

# ---------------- Extracción por año ----------------
batch_size = 20000

for year in anios:
    print(f"\nProcesando año {year}...")

    domain = [
        ['period_id','ilike', str(year)],
        ['state','not in',['draft','cancel','proforma','proforma2']],
        ['type','in',['out_invoice','out_refund']]  # Facturas y notas crédito
    ]

    # Buscar cuántos registros hay
    total = models.execute_kw(
        db, uid, password,
        'account.invoice.report', 'search_count',
        [domain]
    )
    print(f"Total de registros {year}: {total}")

    # Descargar en lotes
    offset = 0
    df_total = pd.DataFrame()

    while True:
        print(f"Cargando registros {year} desde offset {offset}...")
        registros = models.execute_kw(
            db, uid, password,
            'account.invoice.report', 'search_read',
            [domain],
            {'fields': campos, 'limit': batch_size, 'offset': offset}
        )
        if not registros:
            break

        df_parcial = pd.DataFrame(registros)
        df_total = pd.concat([df_total, df_parcial], ignore_index=True)
        offset += batch_size

    print(f"Año {year}: {len(df_total)} registros descargados")

    # Guardar en Excel con fecha/hora de ejecución
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_archivo = f"ventas_{year}_{timestamp}.xlsx"
    df_total.to_excel(nombre_archivo, index=False)
    print(f"Archivo guardado: {nombre_archivo}")

# 4.5 HORAS


Procesando año 2025...
Total de registros 2025: 150348
Cargando registros 2025 desde offset 0...
Cargando registros 2025 desde offset 20000...
Cargando registros 2025 desde offset 40000...
Cargando registros 2025 desde offset 60000...
Cargando registros 2025 desde offset 80000...
Cargando registros 2025 desde offset 100000...
Cargando registros 2025 desde offset 120000...
Cargando registros 2025 desde offset 140000...
Cargando registros 2025 desde offset 160000...
Año 2025: 150387 registros descargados
Archivo guardado: ventas_2025_20250924_084803.xlsx


In [20]:
df_total.sample(5)

,date,price_total,product_cost,id,product_marca
20292,2025-08-06,38700.0,20060.94,1586760,AUT*PARTS
20337,2025-08-06,-94400.0,-42286.93,1587070,DONSSON
21290,2025-08-05,37400.0,10692.27,1585958,DONSSON
8424,2025-09-02,45200.0,30506.74,1605617,BALDWIN
47748,2025-06-18,26600.0,18047.06,1553974,BALDWIN


In [23]:
df_total.groupby("product_marca")["product_cost"].sum()

product_marca
False             -2.490211e+08
                   0.000000e+00
AUT*PARTS          1.640640e+08
BALDWIN            2.293295e+09
DONSSON            1.318126e+09
HIDROCOOL         -4.608254e+05
MOBIL IMPORTADO    2.434978e+07
MOBIL NACIONAL     8.881712e+06
OTROS             -8.184528e+06
PERKINS           -3.374400e+06
PURE GUARD         1.053254e+06
RACOR BRASIL       1.552230e+08
RACOR USA          1.123175e+08
RAMA               2.986876e+05
SUPREME            1.504960e+06
URSA               3.147177e+06
Name: product_cost, dtype: float64